# [Multivariate Time Series Classification with sktime](https://www.sktime.org/en/stable/examples/03_classification_multivariate.html)

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
#!pip install sktime

In [4]:
from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.shapelet_based import MrSEQLClassifier
from sktime.datasets import load_basic_motions
from sktime.transformations.panel.compose import ColumnConcatenator

## Load multivariate time series/panel data

The [data set](http://www.timeseriesclassification.com/description.php?Dataset=BasicMotions) we use in this notebook was generated as part of a student project where four students performaed four activities whilst wearing a smart watch. The watch collects 3D accelerometer and a 3D gyroscope it consists of four classes, which are walking, resting, running and badminton. Participants were required to record motion a total of five times, and the data is sampled once every tenth of a second, for a ten second period.

In [5]:
X, y = load_basic_motions(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60, 6) (60,) (20, 6) (20,)


In [6]:
# multivariate input data
X_train.head()

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5
9,0 -0.407421 1 -0.407421 2 2.355158 3...,0 1.413374 1 1.413374 2 -3.928032 3...,0 0.092782 1 0.092782 2 -0.211622 3...,0 -0.066584 1 -0.066584 2 -3.630177 3...,0 0.223723 1 0.223723 2 -0.026634 3...,0 0.135832 1 0.135832 2 -1.946925 3...
24,0 0.383922 1 0.383922 2 -0.272575 3...,0 0.302612 1 0.302612 2 -1.381236 3...,0 -0.398075 1 -0.398075 2 -0.681258 3...,0 0.071911 1 0.071911 2 -0.761725 3...,0 0.175783 1 0.175783 2 -0.114525 3...,0 -0.087891 1 -0.087891 2 -0.503377 3...
5,0 -0.357300 1 -0.357300 2 -0.005055 3...,0 -0.584885 1 -0.584885 2 0.295037 3...,0 -0.792751 1 -0.792751 2 0.213664 3...,0 0.074574 1 0.074574 2 -0.157139 3...,0 0.159802 1 0.159802 2 -0.306288 3...,0 0.023970 1 0.023970 2 1.230478 3...
7,0 -0.352746 1 -0.352746 2 -1.354561 3...,0 0.316845 1 0.316845 2 0.490525 3...,0 -0.473779 1 -0.473779 2 1.454261 3...,0 -0.327595 1 -0.327595 2 -0.269001 3...,0 0.106535 1 0.106535 2 0.021307 3...,0 0.197090 1 0.197090 2 0.460763 3...
34,0 0.052231 1 0.052231 2 -0.54804...,0 -0.730486 1 -0.730486 2 0.70700...,0 -0.518104 1 -0.518104 2 -1.179430 3...,0 -0.159802 1 -0.159802 2 -0.239704 3...,0 -0.045277 1 -0.045277 2 0.023970 3...,0 -0.029297 1 -0.029297 2 0.29829...


In [11]:
type(X_train['dim_0'].loc[9])

pandas.core.series.Series

This is the structure - each cell is a pandas Series!

In [12]:
type(X_train['dim_1'].loc[9])

pandas.core.series.Series

In [13]:
X_train['dim_1'].loc[9]

0     1.413374
1     1.413374
2    -3.928032
3    -1.480309
4    -2.782044
        ...   
95   -0.390737
96   -0.462142
97    0.216479
98    0.328636
99    0.537231
Length: 100, dtype: float64

In [14]:
# Multi-class target variable
np.unique(y_train)

array(['badminton', 'running', 'standing', 'walking'], dtype=object)

## Multivariate Classification

sktime offers three main ways of solving multivariate time series classification problems:
1. *Concatenation* of time series columns into a single long time series column via `ColumnConcatenator` and apply a classifier to the concatenated data.
2. *Column-wise ensembling* via `ColumnEnsembleClassifier` in which one classifier is fitted for each time series column and their predictions aggregated,
3. *Bespoke estimator-specific methods* for handling multivariate time series data, e.g., finding shapelets in multidimensional spaces.

## Time series concatenation

We can concatenate multivariate time series/panel data into long univariate time series/panel and then apply a classifier to the univariate data.

In [16]:
steps = [
         ("concatenate", ColumnConcatenator()),
         ("classifier", TimeSeriesForestClassifier(n_estimators=100)),
]
clf = Pipeline(steps)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

## Column Ensembling

We can also fit one classifier for each time series column and then aggregate their predictions. The interface is similar to the familiar `ColumnTransformer` from sklearn.

In [18]:
clf = ColumnEnsembleClassifier(
    estimators=[
                ("TSF0", TimeSeriesForestClassifier(n_estimators=100), [0]),
                ("BOSSEnsemble3", BOSSEnsemble(max_ensemble_size=5), [3]),
    ]
)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9

## Bespoke classification algorithms

Another approach is to use bespoke (or classifier-specific) methods for multivariate time series data. Here, we try out the MrSEQL algorithm in multidimensional space.

In [19]:
clf = MrSEQLClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

## [KNeighbors](https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.classification.distance_based.KNeighborsTimeSeriesClassifier.html?highlight=kneighbors)

In [31]:
from sktime.datasets import load_basic_motions
X_train, y_train = load_basic_motions(return_X_y=True, split="train")
X_test, y_test = load_basic_motions(return_X_y=True, split="test")
X_train.head()

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5
0,0 0.079106 1 0.079106 2 -0.903497 3...,0 0.394032 1 0.394032 2 -3.666397 3...,0 0.551444 1 0.551444 2 -0.282844 3...,0 0.351565 1 0.351565 2 -0.095881 3...,0 0.023970 1 0.023970 2 -0.319605 3...,0 0.633883 1 0.633883 2 0.972131 3...
1,0 0.377751 1 0.377751 2 2.952965 3...,0 -0.610850 1 -0.610850 2 0.970717 3...,0 -0.147376 1 -0.147376 2 -5.962515 3...,0 -0.103872 1 -0.103872 2 -7.593275 3...,0 -0.109198 1 -0.109198 2 -0.697804 3...,0 -0.037287 1 -0.037287 2 -2.865789 3...
2,0 -0.813905 1 -0.813905 2 -0.424628 3...,0 0.825666 1 0.825666 2 -1.305033 3...,0 0.032712 1 0.032712 2 0.826170 3...,0 0.021307 1 0.021307 2 -0.372872 3...,0 0.122515 1 0.122515 2 -0.045277 3...,0 0.775041 1 0.775041 2 0.383526 3...
3,0 0.289855 1 0.289855 2 -0.669185 3...,0 0.284130 1 0.284130 2 -0.210466 3...,0 0.213680 1 0.213680 2 0.252267 3...,0 -0.314278 1 -0.314278 2 0.018644 3...,0 0.074574 1 0.074574 2 0.007990 3...,0 -0.079901 1 -0.079901 2 0.237040 3...
4,0 -0.123238 1 -0.123238 2 -0.249547 3...,0 0.379341 1 0.379341 2 0.541501 3...,0 -0.286006 1 -0.286006 2 0.208420 3...,0 -0.098545 1 -0.098545 2 -0.023970 3...,0 0.058594 1 0.058594 2 0.175783 3...,0 -0.074574 1 -0.074574 2 0.114525 3...


In [32]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [33]:
classifier = KNeighborsTimeSeriesClassifier()
classifier.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier()

In [34]:
y_pred = classifier.predict(X_test)

In [35]:
y_pred

array(['standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'running', 'running', 'running', 'running', 'running', 'running',
       'running', 'running', 'running', 'running', 'walking', 'walking',
       'walking', 'walking', 'walking', 'walking', 'walking', 'walking',
       'walking', 'walking', 'badminton', 'badminton', 'badminton',
       'badminton', 'badminton', 'badminton', 'badminton', 'badminton',
       'walking', 'badminton'], dtype='<U9')

## Demo of [ROCKET Transform](https://www.sktime.org/en/stable/examples/rocket.html)

In [36]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline

from sktime.datasets import load_basic_motions # multivariate dataset
from sktime.transformations.panel.rocket import Rocket

### 3.1 Load the Training Data

In [37]:
X_train, y_train = load_basic_motions(split="train", return_X_y=True)

In [38]:
X_train.head()

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5
0,0 0.079106 1 0.079106 2 -0.903497 3...,0 0.394032 1 0.394032 2 -3.666397 3...,0 0.551444 1 0.551444 2 -0.282844 3...,0 0.351565 1 0.351565 2 -0.095881 3...,0 0.023970 1 0.023970 2 -0.319605 3...,0 0.633883 1 0.633883 2 0.972131 3...
1,0 0.377751 1 0.377751 2 2.952965 3...,0 -0.610850 1 -0.610850 2 0.970717 3...,0 -0.147376 1 -0.147376 2 -5.962515 3...,0 -0.103872 1 -0.103872 2 -7.593275 3...,0 -0.109198 1 -0.109198 2 -0.697804 3...,0 -0.037287 1 -0.037287 2 -2.865789 3...
2,0 -0.813905 1 -0.813905 2 -0.424628 3...,0 0.825666 1 0.825666 2 -1.305033 3...,0 0.032712 1 0.032712 2 0.826170 3...,0 0.021307 1 0.021307 2 -0.372872 3...,0 0.122515 1 0.122515 2 -0.045277 3...,0 0.775041 1 0.775041 2 0.383526 3...
3,0 0.289855 1 0.289855 2 -0.669185 3...,0 0.284130 1 0.284130 2 -0.210466 3...,0 0.213680 1 0.213680 2 0.252267 3...,0 -0.314278 1 -0.314278 2 0.018644 3...,0 0.074574 1 0.074574 2 0.007990 3...,0 -0.079901 1 -0.079901 2 0.237040 3...
4,0 -0.123238 1 -0.123238 2 -0.249547 3...,0 0.379341 1 0.379341 2 0.541501 3...,0 -0.286006 1 -0.286006 2 0.208420 3...,0 -0.098545 1 -0.098545 2 -0.023970 3...,0 0.058594 1 0.058594 2 0.175783 3...,0 -0.074574 1 -0.074574 2 0.114525 3...


### 3.2 Initialize ROCKET and Transform the Training Data

In [39]:
rocket = Rocket()
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)

In [41]:
X_train.shape

(40, 6)

In [40]:
X_train_transform.shape

(40, 20000)

### 3.3 Fit a Classifier

In [42]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

### 3.4 Load and Transform the Test Data

In [43]:
X_test, y_test = load_basic_motions(split="test", return_X_y=True)
X_test_transform = rocket.transform(X_test)

### 3.5 Classify the Test Data

In [44]:
classifier.score(X_test_transform, y_test)

1.0

### Pipeline Example

In [45]:
rocket_pipeline = make_pipeline(
    Rocket(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
)

# It is necessary to pass y_train to the pipeline
# y_train is not used for the transform, but it is used by the classifier
rocket_pipeline.fit(X_train, y_train)

Pipeline(steps=[('rocket', Rocket()),
                ('ridgeclassifiercv',
                 RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                                   normalize=True))])

In [46]:
# Load and classify the test data
rocket_pipeline.score(X_test, y_test)

1.0

## [MiniRocket](https://github.com/alan-turing-institute/sktime/blob/main/examples/minirocket.ipynb)

MiniRocket transforms input time series using a small, fixed set of convolutional kernels. MiniRocket uses PPV pooling to compte a single feature for each of the resulting feature maps (i.e., the proportion of positive values). The transformed features are used to train a linear classifier.

MiniRocket and MiniRocketMultivariate are compiled by Numba on import. The compiled functions are cached, so this should only happen once (i.e., the first time you import MiniRocket or MiniRocketMultivariate).

Note that shorter time series can be padded with `PaddingTransformer (sktime.transformers.panel.padder)`

In [62]:
#!pip install --upgrade numba

In [63]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline

from sktime.datasets import load_basic_motions
from sktime.transformations.panel.rocket import MiniRocketMultivariate

In [66]:
X_train, y_train = load_basic_motions(split="train", return_X_y=True)
minirocket_multi = MiniRocketMultivariate()
minirocket_multi.fit(X_train)
X_train_transform = minirocket_multi.transform(X_train)
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)
X_test, y_test = load_basic_motions(split="test", return_X_y=True)
X_test_transform = minirocket_multi.transform(X_test)
classifier.score(X_test_transform, y_test)

1.0

## [Mr-SEQL](https://www.sktime.org/en/stable/examples/mrseql.html)

Mr-SEQL also support multivariate time series. Mr-SEQL extracts features from each dimension of the data independently.



In [51]:
from sklearn import metrics

In [52]:
X, y = load_basic_motions(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60, 6) (60,) (20, 6) (20,)


In [54]:
ms = MrSEQLClassifier()

# fit training data
ms.fit(X_train, y_train)

predicted = ms.predict(X_test)

# Classification accuracy
print(f"Accuracy with mr-seql: {metrics.accuracy_score(y_test, predicted)}")

Accuracy with mr-seql: 1.0


In [58]:
from sktime.datasets import load_arrow_head

In [59]:
train_x, train_y = load_arrow_head(split="train", return_X_y=True)

In [60]:
train_x.head()

,dim_0
0,0 -1.9630 1 -1.9578 2 -1.9561 3 ...
1,0 -1.7746 1 -1.7740 2 -1.7766 3 ...
2,0 -1.8660 1 -1.8420 2 -1.8350 3 ...
3,0 -2.0738 1 -2.0733 2 -2.0446 3 ...
4,0 -1.7463 1 -1.7413 2 -1.7227 3 ...


## [Interval based time series classification in sktime](https://github.com/alan-turing-institute/sktime/blob/main/examples/interval_based_classification.ipynb)

Interval-based approaches look at phase dependent intervals of the full series, calculating summary statistics from selected subseries to be used in classification. Canonical Interval Forest (CIF) and Diverse Representation Canonical Interval Forest (DrCIF) have multivariate capabilities.

In [67]:
from sklearn import metrics
from sktime.classification.interval_based import (
    CanonicalIntervalForest,
    DrCIF
)
from sktime.datasets import load_basic_motions

In [68]:
X_train_mv, y_train_mv = load_basic_motions(split="train", return_X_y=True)
X_test_mv, y_test_mv = load_basic_motions(split="test", return_X_y=True)

### Canonical Interval Forest (CIF)

In [70]:
cif_m = CanonicalIntervalForest(n_estimators=50, att_subsample_size=8, random_state=47)
cif_m.fit(X_train, y_train)

cif_m_preds = cif_m.predict(X_test)
print(f"CIF Accuracy: {metrics.accuracy_score(y_test, cif_m_preds)}")

CIF Accuracy: 1.0


### Diverse Representation Canonical Interval Forest (DrCIF)

In [71]:
drcif_m = DrCIF(n_estimators=50, att_subsample_size=10, random_state=47)
drcif_m.fit(X_train, y_train)

drcif_m_preds = drcif_m.predict(X_test)
print(f"DrCIF Accuracy: {metrics.accuracy_score(y_test, drcif_m_preds)}")

DrCIF Accuracy: 1.0


## [Dictionary-based time series classification in sktime](https://github.com/alan-turing-institute/sktime/blob/main/examples/dictionary_based_classification.ipynb)

WEASEL has a multivariate extension called MUSE and TDE has multivariate capabilities.

In [73]:
from sktime.classification.dictionary_based import (
    MUSE,
    TemporalDictionaryEnsemble
)

from sktime.datasets import load_basic_motions

X_train_mv, y_train_mv = load_basic_motions(split="train", return_X_y=True)
X_test_mv, y_test_mv = load_basic_motions(split="test", return_X_y=True)

### WEASEL+MUSE (multivariable Symbolic Extension) is the multivariate extension of WEASEL.

In [74]:
muse = MUSE()
muse.fit(X_train_mv, y_train_mv)
muse_preds = muse.predict(X_test_mv)
print(f"MUSE Accuracy: {metrics.accuracy_score(y_test_mv, muse_preds)}")

MUSE Accuracy: 1.0


### Temporal Dictionary Ensemble (TDE)

In [77]:
# Recommended
tde_mv = TemporalDictionaryEnsemble(
    n_parameter_samples=250,
    max_ensemble_size=50,
    randomly_selected_params=50,
    random_state=47,
)

# TDE with a 1 minute build time contract
# tde_m = TemporalDictionaryEnsemble(time_limit_in_minutes=1,
#                                   max_ensemble_size=50,
#                                   randomly_selected_params=50,
#                                   random_state=47)

tde_mv.fit(X_train_mv, y_train_mv)

tde_mv_preds = tde_mv.predict(X_test_mv)
print(f"TDE Accuracy: {metrics.accuracy_score(y_test_mv, tde_mv_preds)}")

TDE Accuracy: 1.0


## [catch22](https://github.com/alan-turing-institute/sktime/blob/main/examples/catch22.ipynb)



In [2]:
#!pip install sktime[all_extras]
!pip install esig

     |████████████████████████████████| 6.9 MB 7.4 MB/s 


In [3]:
from sklearn import metrics
from sktime.classification.feature_based import Catch22Classifier
from sktime.datasets import load_basic_motions
from sktime.transformations.panel.catch22 import Catch22

In [4]:
BM_X_train, BM_y_train = load_basic_motions(split="train", return_X_y=True)
BM_X_test, BM_y_test = load_basic_motions(split="test", return_X_y=True)

In [5]:
c22_mv = Catch22()
c22_mv.fit(BM_X_train, BM_y_train)

Catch22()

In [6]:
transformed_data_mv = c22_mv.transform(BM_X_train)
print(transformed_data_mv.head())

         0         1     2         3   ...        18        19        20    21
0 -0.208846  0.136909   9.0 -0.510833  ...  0.659574  0.127660  0.002256   9.0
1  0.739665  0.144455  79.0 -0.520000  ...  0.872340  0.170213  0.014384  10.0
2  0.008406  0.199387  25.0 -0.446667  ...  0.659574  0.127660  0.000917  11.0
3  0.090501 -0.033292  17.0 -0.530000  ...  0.638298  0.127660  0.005236   8.0
4 -0.096920 -0.009260  26.0 -0.513333  ...  0.617021  0.787234  0.011036  10.0

[5 rows x 22 columns]
